In [ ]:
import org.apache.spark.streaming.StreamingContext._
import org.apache.spark.streaming.{Seconds, StreamingContext}
import org.apache.spark.{SparkConf, SparkContext}
import org.apache.spark.sql.SQLContext
import org.apache.spark.streaming.Duration
import org.apache.spark.sql.functions.udf
import org.apache.spark.sql.types
import spark.implicits._

In [ ]:
!rm -r checkpoint


In [ ]:
!mkdir checkpoint

In [ ]:
val schema = spark.read.json("input/*.json").schema

In [ ]:
val df = spark.readStream
    .format("json")
    .schema(schema)
    .option("multiLine", "false")
    .load("input/*.json")
    .select("*", "_metadata")

/*
val df = spark.readStream
    .format("json")
    .schema(schema)
    .option("multiLine", "false")
    .option("cleanSource", "archive")
    .option("sourceArchiveDir", f"input_archive")
    .load("input/*.json")
    .select("*", "_metadata")
*/

In [ ]:
val df2 = df.writeStream
    .option("checkpointLocation", "checkpoint")
    .option("overwriteSchema",true)
    .trigger(org.apache.spark.sql.streaming.Trigger.ProcessingTime("1 second"))
    .toTable("houses")

In [ ]:
df2.processAllAvailable
//df2.stop
df2.awaitTermination

In [ ]:
val df3 = spark.read.format("jdbc").table("houses")

In [ ]:
df3.count

In [ ]:
df2.stop

In [ ]:
df3.select("_metadata").distinct.show(truncate=false)

In [ ]:

val deltaTable = DeltaTable.forName(spark, "data")

deltaTable.as("data")
  .merge(
    people_10m_updates.as("people_10m_updates"),
    "data.id = people_10m_updates.id"
  )
  .whenMatched()
  .updateAll()
  .whenNotMatched()
  .insertAll()
  .execute()